# Linear Regression

## Introduction
Regression problems study and estimate the relationship between some features from different observations and their (continuous) responses. Linear regression is a simple approach that is useful for regression problems. The case of one explanatory feature is called simple linear regression. For example, given the size of a house, we want to predict its price. When there also exist other explanatory features, like locations, high, low temperatures of the year, etc., the process is called multiple linear regression. 

Linear regression is very useful in machine learning and data analytics since it is:
* easy and runs fast
* parameter free
* intepretable
* foundation of many other methods

We will also cover logistic regression in [next chapter](logistic_regression.ipynb). 

## Model specification
In linear regression, the relationships are modeled using ***linear predictor functions*** whose unknown model parameters are estimated from the data. Formally, suppose $X_1,X_2,\cdots,X_k$ are $k$ independent explanary variables (features) and $y$ is the response, we would like to learn a model $\boldsymbol {\beta}$, such that $y=\beta_0+\beta_1X_1+\cdots+\beta_kX_k+\varepsilon$. 

Now given $n$ observations of the above $k$ features, denoted by $ \{x_{i1},\ldots ,x_{ik}\}_{i=1}^{n}$ and their responses $\{y_i\}_{i=1}^n$, the relationship is modeled through a disturbance term or error variable $\varepsilon_i$ — an unobserved random variable that adds noise to the linear relationship between the dependent variable and regressors. Thus the model takes the form
$$y_{i}=\beta _{1}x_{i1}+\cdots +\beta _{k}x_{ik}+\varepsilon _{i}=\mathbf {x} _{i}^{\rm {T}}{\boldsymbol {\beta }}+\varepsilon _{i},\qquad i=1,\ldots ,n,$$

Therefore, to find the best linear function that fits the data is to minimize the error $\varepsilon$. We optimize the mean square loss function $l(\boldsymbol{\beta})$
$$l(\boldsymbol {\beta})=\frac{1}{n}\sum_{i=1}^n |\varepsilon_i|^2=\frac{1}{n}\|\boldsymbol {\varepsilon}\|_2^2=\frac{1}{n}\|\mathbf {y}-\mathbf {X}\boldsymbol{\beta}\|_2^2 $$
, where 
$$\mathbf {y} ={\begin{pmatrix}y_{1}\\y_{2}\\\vdots \\y_{n}\end{pmatrix}},\quad 
{\displaystyle \mathbf {X} ={\begin{pmatrix}\mathbf {x} _{1}^{\rm {T}}\\\mathbf {x} _{2}^{\rm {T}}\\\vdots \\\mathbf {x} _{n}^{\rm {T}}\end{pmatrix}}={\begin{pmatrix}1 & x_{11}&\cdots &x_{1k}\\1 & x_{21}&\cdots &x_{2k}\\ \vdots & \vdots &\ddots &\vdots \\1 & x_{n1}&\cdots &x_{nk}\end{pmatrix}}}, \quad 
{\displaystyle {\boldsymbol {\beta }}={\begin{pmatrix}\beta_{0}\\ \beta _{1}\\\vdots \\\beta _{k}\end{pmatrix}},\quad {\boldsymbol {\varepsilon }}={\begin{pmatrix}\varepsilon _{1}\\\varepsilon _{2}\\\vdots \\\varepsilon _{n}\end{pmatrix}}.}$$

Notice the minimizer of $l(\beta)$ has close form solution: $\boldsymbol{\beta}^*=(\mathbf {X} ^{\rm {\top}}\mathbf {X} )^{-1}\mathbf {X} ^{\rm {\top}}\mathbf {y}$. However, computing the inverse of an $n\times n$ matrix requires $\mathcal {O}(n^3)$ time complexity and is not ideal for large $n$. Therefore we usually conduct gradient descent algorithm to optimize $l(\boldsymbol {\beta})$.


## Data handling
Here is an example of a statistical relationship. The response variable $y$ is the mortality due to skin cancer (number of deaths per 10 million people) and the predictor variable $\mathbf X$ is the latitude (degrees North) at the center of each of 49 states in the U.S. ([skincancer.txt](/files/skincancer.txt)) (The data were compiled in the 1950s, so Alaska and Hawaii were not yet states. And, Washington, D.C. is included in the data set even though it is not technically a state.)



In [1]:
import mxnet as mx
def getData(fname):
    data=[]
    label=[]
    firstline=True
    for line in file(fname):
        if (firstline):
            firstline=False
            continue
        tks = line.strip().split()
        data.append([float(tks[1]),int(tks[3]),float(tks[4])]) # omit the first column of user ID.
        label.append(int(tks[2]))
    return mx.nd.array(data),mx.nd.array(label)
X_train,y_train=getData('skincancer.txt')
X_train.shape

(49L, 3L)

For simpler calculations, we add an all one feature to each sample to avoid the use of constant: $\beta^T x + b \Rightarrow [\beta, b]^T[x,1]$. MXNet NDArray has the concatenate attribute designed for this task.

We reshape the labels "y_train" as a nx1 2D array for the sake of mxnet ndarray multiplication. (Otherwise y_train will only have one dimension and won't be able to multiply with a 2D NDArray)

In [2]:
n,p = X_train.shape
n,p
n=int(n)
p=int(p)
X_train=mx.nd.concatenate([X_train.T,mx.nd.ones((1,n))]).T # add the terms for constant
p+=1
n,p
y_train=y_train.reshape((n,1))

## Optimization

### Step-by-step implementation by NDArray:
We first define the mean square error:


In [3]:
def MSE(label, pred):
    return mx.nd.norm(label-pred).asscalar()**2/n

In order to conduct gradient descent on $l(\boldsymbol{\beta})$, we compute its gradient $\mathbf g$ first:
$\mathbf g\equiv\nabla_{\boldsymbol{\beta}}l(\boldsymbol{\beta})=\mathbf{X}^{\top}(\mathbf{X}\boldsymbol{\beta}-\mathbf{y})$.

With the help of MXNet NDArray, we could compute the gradience step by step as follows: 

In [4]:
# initialization
beta=mx.nd.zeros((p,1))
# multiplication
predy=mx.nd.dot(X_train,beta)
g=mx.nd.dot(X_train.T,predy-y_train)*2./n
g

<NDArray 4x1 @cpu(0)>

In order to avoid overfitting, we add an $\ell_2$ regularization $\alpha\|\boldsymbol{\beta}\|_2^2$ to the loss function $l(\boldsymbol{\beta})$, therefore in the gradient descent step, we do:
$\boldsymbol{\beta}\leftarrow \boldsymbol{\beta}-2\alpha\boldsymbol{\beta}-g=(1-2\alpha)\boldsymbol{\beta}-\eta \mathbf g$, where $\eta$ is the learning rate.

Therefore the whole training process of logistic regression becomes:

In [5]:
def train(X_train, y_train,itr,alpha,learning_rate):
    beta=mx.nd.zeros((p,1))
    predy=mx.nd.dot(X_train,beta)
    for i in range(itr):
        g=mx.nd.dot(X_train.T,predy-y_train)*2./n
        beta=beta*(1-2*alpha)-learning_rate*g
        predy=mx.nd.dot(X_train,beta)
        print "iter %d: mean square error=: %.4f"%(i, MSE(predy,y_train))
    return beta
beta=train(X_train,y_train,20,0.01,2e-5)

iter 0: mean square error=: 9987.7183
iter 1: mean square error=: 4923.6607
iter 2: mean square error=: 3104.3352
iter 3: mean square error=: 2424.0207
iter 4: mean square error=: 2155.1912
iter 5: mean square error=: 2041.4630
iter 6: mean square error=: 1989.6622
iter 7: mean square error=: 1964.3709
iter 8: mean square error=: 1951.2951
iter 9: mean square error=: 1944.2423
iter 10: mean square error=: 1940.3246
iter 11: mean square error=: 1938.1049
iter 12: mean square error=: 1936.8280
iter 13: mean square error=: 1936.0852
iter 14: mean square error=: 1935.6467
iter 15: mean square error=: 1935.3831
iter 16: mean square error=: 1935.2209
iter 17: mean square error=: 1935.1173
iter 18: mean square error=: 1935.0475
iter 19: mean square error=: 1934.9980


### Symbol manipulation
Besides the tensor computation interface NDArray, another important object in MXNet is Symbol provided by mxnet.symbol, or mxnet.sym for short.

A symbol represents a multi-output symbolic expression. They are composited by operators, such as simple matrix operations (e.g. “+”), or a neural network layer (e.g. convolution layer). Details can be found in http://mxnet.io/tutorials/python/symbol.html.

Training by using NDArray is straightforward, and can support step-by-step debug. However, mxnet.symbol not only inherits almost all functionalities of NDArray, but also provides automatic differentiation and is easy for the backend to optimize the computation and memory usage. We will show the advantages in the following implementations.

Firstly, we show how symbol defines the computations first and can be easily bind with data afterwards. To finish this task, we could simply copy the function "train", but only change all the "mx.nd" to "mx.sym". Notice mxnet.symbol inherits the operations supported by mx.ndarray, and functions in a symbolic way. 

In [6]:
def train(X_train, y_train):
    beta=mx.sym.zeros((p,1))
    predy=mx.sym.dot(X_train,beta)
    itr=20
    learning_rate=2e-5
    alpha=0.01
    for i in range(itr):
        g=mx.sym.dot(mx.sym.transpose(X_train),predy-y_train)*2./n
        beta=beta*(1-2*alpha)-learning_rate*g
        predy=mx.sym.dot(X_train,beta)
    return beta

#### Bind with data and evaluate:
After defining the train function, we can run it and get a symbolic output.
Notice this whole process only declares computation, we need to bind with data to run.
In the following, we bind with the same data as before and get the same result we got using NDArray: 

In [7]:
X_train_sym = mx.sym.Variable('X_train_sym')
y_train_sym = mx.sym.Variable('y_train_sym')
beta_sym=train(X_train_sym,y_train_sym)
ex=beta_sym.bind(ctx=mx.cpu(), args={'X_train_sym':X_train, 'y_train_sym':y_train})
ex.forward()
beta=ex.outputs[0]
mse=MSE(y_train, mx.nd.dot(X_train,beta))
print "Final mean square error: %.4f"%(mse)

Final mean square error: 1934.9980


### Module auto differentiation
Instead of computing the gradient explicitly, mxnet.symbol actually supports auto differentiation. 
We first define a single layered network with linear regression output, which is equivalent to linear regression, and show how mxnet.mod.Module supports train, predict and evaluate automatically. 

In [8]:
import mxnet as mx
net = mx.sym.Variable('data')
net = mx.sym.FullyConnected(data=net, name='fc1', num_hidden=1)
net = mx.sym.LinearRegressionOutput(data=net, name='softmax')
print(net.list_arguments())
print(net.list_outputs())

['data', 'fc1_weight', 'fc1_bias', 'softmax_label']
['softmax_output']


Most training and inference modules in MXNet accepts data iterators. MXNet.io.NDArray converts training data and labels to data iterators which could be bind with the module. The following codes implement a simplified training procedure by doing stochastic gradient descent on one pass over the data. It shows how MXnet.mod.Module automatically supports gradient computation and model update procedures.

In [9]:
mod = mx.mod.Module(symbol=net)
# mxnet.io.NDArrayIter takes features and labels 
train_iter=mx.io.NDArrayIter(data=X_train,label=y_train,batch_size=1)
print(train_iter.provide_data)
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
# init parameters
mod.init_params(initializer=mx.init.Xavier(magnitude=0.))

# init optimizer
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 2e-5), ))

# use mean square error as the metric
metric = mx.metric.create('mse')

# train one epoch, i.e. going over the data iter one pass
for batch in train_iter:
    mod.forward(batch, is_train=True)       # compute predictions
    mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
    mod.backward()                          # compute gradients
    mod.update()                            # update parameters using SGD
    
# training error
print(metric.get())

[DataDesc[data,(1, 4L),<type 'numpy.float32'>,NCHW]]
('mse', 3763.6952887067991)


### Module training

Finally we show how [Module](http://mxnet.io/tutorials/python/module.html) simplifies the whole procedures above by using DataIter, especially when reading large datasets from filesystems. 

MXNet Module provides high-level APIs for training, predicting and evaluating. To fit a module, we directly call the fit function with the input DataIters.

In [10]:
mod = mx.mod.Module(symbol=net)
mod.fit(train_iter, eval_data=train_iter, optimizer='sgd',optimizer_params={'learning_rate':2e-5},eval_metric='mse',num_epoch=20)
for batch in train_iter:
    mod.update_metric(metric,batch.label)  #evaluate the mean square error
predy=mod.predict(train_iter) # prediction
print(metric.get())
print(predy)

('mse', 2440.7994793410203)
<NDArray 49x1 @cpu(0)>
